In [25]:
import tensorflow as tf
import pandas as pd
tf.__version__

'1.13.1'

In [28]:
mnist = tf.contrib.learn.datasets.mnist.read_data_sets(train_dir='./logdir_data', one_hot=True)
df_test = pd.read_csv('../test.csv')


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./logdir_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./logdir_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Ext

In [39]:
def conv_layer(input, channels_in, channels_out, name="conv"):
    with tf.name_scope(name):
        w = tf.Variable(tf.zeros([5,5, channels_in, channels_out]))
        b = tf.Variable(tf.zeros([channels_out]))
        conv = tf.nn.conv2d(input, w, strides=[1, 1, 1, 1], padding="SAME")
        act = tf.nn.relu(conv + b)
        return act

In [40]:
def fc_layer(input, channels_in, channels_out, name="fc"):
    with tf.name_scope(name):
        w = tf.Variable(tf.zeros([channels_in, channels_out]))
        b = tf.Variable(tf.zeros([channels_out]))
        act = tf.nn.relu(tf.matmul(input, w) + b)
        return act

### Setup placeholders and reshape data

In [41]:
x = tf.placeholder(tf.float32, shape=[None, 784], name='x')
y = tf.placeholder(tf.float32, shape=[None, 10], name='labels')
x_image = tf.reshape(x, [-1, 28, 28, 1])

### Create network

In [60]:
#conv1 = conv_layer(x_image, 1, 32, "conv1")
#pool1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

#conv2 = conv_layer(pool1, 32, 64, "conv2")
#pool2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
#flattened = tf.reshape(x_image, [-1, 7 * 7 * 64])

fcl = fc_layer(x, 784, 32, "fcl")
#logits = fc_layer(fcl, 1024, 10, "fc2")

### Loss & training

In [61]:
with tf.name_scope("xent"):
    xent = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))

with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

#compute accuracy
with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)) #?


In [62]:
sess = tf.Session()
# tf.summary.Filwriter('folder to write to') a tensorflow class that writes data for Tensorboard
writer = tf.summary.FileWriter("./logdir")
writer.add_graph(sess.graph)

### Train Model

In [63]:
# Initialize variables
sess.run(tf.global_variables_initializer())

# Train for 2000 steps
for i in range(2000):
    batch = mnist.train.next_batch(100)
    if i % 500 == 0:
        [train_accuracy] = sess.run([accuracy], feed_dict={x: batch[0], y: batch[1]})
        print("step %d, training accuracy %g" % (i, train_accuracy))
    
    # Run the training step
    sess.run(train_step, feed_dict={x: batch[0], y: batch[1]})

step 0, training accuracy 0.05


InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder' with dtype float and shape [?,784]
	 [[node Placeholder (defined at <ipython-input-9-36c80cc94596>:1) ]]

Caused by op 'Placeholder', defined at:
  File "/home/josecyc/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/josecyc/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/josecyc/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/josecyc/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/josecyc/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 497, in start
    self.io_loop.start()
  File "/home/josecyc/anaconda3/lib/python3.6/site-packages/tornado/platform/asyncio.py", line 132, in start
    self.asyncio_loop.run_forever()
  

### Visualizing graph